In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.37.0 to work with dp100-workspace


In [2]:
from azureml.core import Experiment
from azureml.core import Model
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
import pandas as pd
import numpy as np
import joblib
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = "car-training")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
car = pd.read_csv('data2/car-prediction2.csv')

# applying label encoding
pre = preprocessing.LabelEncoder()
data = car.apply(pre.fit_transform)

#Normalize the numeric columns
scaler = MinMaxScaler()
num_cols = ['Selling_Price','Present_Price','Kms_Driven','car_age']
data[num_cols] = scaler.fit_transform(data[num_cols])

# Separate features and labels
x, y = data[['Selling_Price','Present_Price','Kms_Driven','Fuel_Type','Seller_Type','Transmission','car_age']].values, data['Owner'].values

# Split data into training set and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

# training the model
print('training Logistic regression model')

reg= 0.02

model=LogisticRegression(C=1/reg,solver="liblinear").fit(x_train,y_train)
# model = DecisionTreeClassifier().fit(x_train, y_train)
# print('training Logistic regression model')
# model=LogisticRegression(C=1/reg,solver="liblinear").fit(x_train,y_train)


# calculate accuracy
y_hat = model.predict(x_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(x_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'car_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/car_model.pkl', model_name='car_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

Starting experiment: car-training
Loading Data...
training Logistic regression model
Accuracy: 0.9180327868852459
AUC: 0.6785714285714286
Model trained and registered.


In [3]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

car_model version: 18
	 Training context : Inline Training
	 AUC : 0.6785714285714286
	 Accuracy : 0.9180327868852459


car_model version: 17
	 Training context : Inline Training
	 AUC : 0.6785714285714286
	 Accuracy : 0.9180327868852459


car_model version: 16
	 Training context : Inline Training
	 AUC : 0.6785714285714286
	 Accuracy : 0.9180327868852459


car_model version: 15
	 Training context : Inline Training
	 AUC : 0.6785714285714286
	 Accuracy : 0.9180327868852459


car_model version: 14
	 Training context : Inline Training
	 AUC : 0.5
	 Accuracy : 0.9180327868852459


car_model version: 13
	 Training context : Inline Training
	 AUC : 0.5
	 Accuracy : 0.9180327868852459


diabetes_model version: 9
	 Training context : Inline Training
	 AUC : 0.8806209264142494
	 Accuracy : 0.8946666666666667


car_model version: 12
	 Training context : Inline Training
	 AUC : 0.5
	 Accuracy : 0.9180327868852459


car_model version: 11
	 Training context : Inline Training
	 AUC : 0.692857142857

In [4]:
model = ws.models['car_model']
print(model.name, 'version', model.version)

car_model version 18


In [5]:
# Deploying Model as WebService

import os

folder_name = 'car-deploy2'

# creating a folder for webservice
experiment_folder = './' + folder_name
os.makedirs(experiment_folder,exist_ok=True)
print(folder_name, 'folder created.')

# score script
script_file = os.path.join(experiment_folder,"score_car.py")

car-deploy2 folder created.


In [6]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('car_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-sold', 'sold']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)


Overwriting ./car-deploy2/score_car.py


In [7]:
# creating enviroment

from azureml.core.conda_dependencies import CondaDependencies

myenv=CondaDependencies()
myenv.add_conda_package('scikit-learn')

env_file = os.path.join(experiment_folder,"car_env.yml")

with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print('Saved dependency info in',env_file)

# print yml file
with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in ./car-deploy2/car_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



In [8]:
# Creating endpoint for deploying service

from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "car-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-01-21 09:04:57+00:00 Creating Container Registry if not exists.
2022-01-21 09:04:57+00:00 Registering the environment.
2022-01-21 09:05:00+00:00 Use the existing image.
2022-01-21 09:05:01+00:00 Generating deployment configuration.
2022-01-21 09:05:02+00:00 Submitting deployment to compute.
2022-01-21 09:05:05+00:00 Checking the status of deployment car-service..
2022-01-21 09:06:38+00:00 Checking the status of inference endpoint car-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [9]:
# checking health of service

print(service.state)
print(service.get_logs())

Healthy
2022-01-21T09:06:29,624132800+00:00 - iot-server/run 
2022-01-21T09:06:29,625637400+00:00 - rsyslog/run 
2022-01-21T09:06:29,627937200+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-01-21T09:06:29,685068100+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-01-21T09:06:29,997909200+00:00 - iot-server/finish 1 0
2022-01-21T09:06:30,006605300+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (68)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 100
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2022-01-21 09:06:32,617 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2022-01-21 09:06:32,618 | root | INFO | Starting up request id generator
2022-01-21 09:06:32,618 | root | IN

In [11]:
# Using the webservice

import json

# x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
x_new = [[0.38,0.38,0.42,2,0,1,0.27]]
print ('car: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

car: [0.38, 0.38, 0.42, 2, 0, 1, 0.27]
not-sold
